In [256]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(20,15),"font.size":20,"axes.titlesize":20,"axes.labelsize":20})

from IPython.display import display
import math
import re

### In this notebook we want to preprocess data extracted from etlib to dataframe with the following features:
* #### problem
* #### solution
* #### brand of a car (if it is known/not generic)
* #### model of a car
* #### a car's year of production

Pairs problem-solution we will use for syntactic analysis (to define what problem is and what solution is) and for NER (to use entities in knowledge graph).</br>
Brand, model and year together we also can use as entities.</br>
For every feature we will perform statistical analysis.</br>

TODO: Maybe we should add weight feature too.

In [257]:
df=pd.read_csv("..\\data\\raw\\etlib_with_index.csv")
df

,text,answer-items-text,answer-items-username,answer-items-date,title,carinfo,breadcrumbs,ctime,curl,id
0,Здравствуйте. В общем день не задался. Ехал со...,['\n ...,"['Het', 'Михаил С.', 'Анатолий.sib']",['\n \n 7.05.20...,Попал в яму на Лада Приора,ВАЗ (Lada) Priora 2007 — н.в.,"['', 'Все вопросы', 'ВАЗ (Lada)', 'Priora', 'Х...",1587739825,https://etlib.ru/qa/popal-v-yamu-46764,0
1,"Уважаемые специалисты подскажите пожалуйста, м...",['\n ...,"['b.k2015', 'leonk51', 'Анатолий.sib', 'b.k201...",['\n \n 20.04.20...,Как подключить кнопку старт на Мазда 323 БФ?,Mazda 323 III (BF) 1985 — 1989,"['', 'Все вопросы', 'Mazda', '323', '323 III (...",1587739827,https://etlib.ru/qa/knopka-start-60350,1
2,Нужен артикул запчасти ремкомплект заднего лев...,['\n ...,"['Автомобиль Аккаунт', 'Анатолий.sib']",['\n \n 7.05.20...,Какой артикул ремкомплекта заднего суппорта Сс...,SsangYong Actyon II 2010 — н.в.,"['', 'Все вопросы', 'SsangYong', 'Actyon', 'Ac...",1587739827,https://etlib.ru/qa/rk-zadnego-supporta-46763,2
3,Не горят половина контрольных ламп. Менял щито...,['\n ...,"['lapin196@mail.ru', 'Михаил С.']",['\n \n 7.05.20...,Не горят контрольные лампы в Шевроле Авео,Chevrolet Aveo I (T200/T250) 2003 — 2012,"['', 'Все вопросы', 'Chevrolet', 'Aveo', 'Aveo...",1587739828,https://etlib.ru/qa/ne-goryat-kontrolnye-lampy...,3
4,Здравствуйте! Как проверить уровень жидкости в...,['\n ...,"['ИЛЬНАР ШАГАЛИЕВ', 'Анатолий.sib']",['\n \n 7.05.20...,Как проверить уровень в вариаторе JFO15E Нисса...,Nissan Qashqai I 2007 — 2014,"['', 'Все вопросы', 'Nissan', 'Qashqai', 'Qash...",1587739829,https://etlib.ru/qa/proverka-urovnya-zhidkosti...,4
...,...,...,...,...,...,...,...,...,...,...
46298,Де знаходиться кнопка та шо отключає подачу па...,['\n ...,"['Дімка323', 'Михаил С.']",['\n \n 24.09.20...,Де кнопка аварійного відключення палива у Рено...,Renault Laguna III 2007 — 2015,"['', 'Все вопросы', 'Renault', 'Laguna', 'Lagu...",1587828498,https://etlib.ru/qa/de-znahoditsya-knopka-49220,46298
46299,Не включается вентилятор при нагреве двигателя,['\n ...,"['Мила (Эм)', 'Михаил С.']",['\n \n 24.09.20...,Не включается вентилятор в Опель Кадет Е,Opel Kadett E 1984 — 1994,"['', 'Все вопросы', 'Opel', 'Kadett', 'Kadett ...",1587828501,https://etlib.ru/qa/sistema-ohlazhdeniya-49219,46299
46300,Добрый день.\n\nПодыскиваю себе пару передних ...,['\n ...,"['savak', 'Анатолий.sib', 'Михаил С.']",['\n \n 24.09.20...,Стоит ли ставить тормозные диски TRW DF4432S н...,Honda Accord VII 2002 — 2007,"['', 'Все вопросы', 'Honda', 'Accord', 'Accord...",1587828502,https://etlib.ru/qa/tormoznye-diski-49217,46300
46301,Почему на панели где показывает скорости моргает?,"['Что мигает?', 'Что там на панели моргает?', ...","['elefant4', 'Михаил С.', 'Михаил С.', 'Анатол...",['\n \n 19.09.20...,На панели где показывает передачи моргает в Хо...,Honda Civic VIII 2006 — 2012,"['', 'Все вопросы', 'Honda', 'Civic', 'Civic V...",1587828503,https://etlib.ru/qa/pocemu-na-poneli-gde-pokaz...,46301


<h3>We want to get pairs problem-solution. Our problem could be in the "title" or in the "text" column (or both) and solution could be in the "comment_texts" column.

In [258]:
df_processed=df.iloc[:,[4,0,1]]
df_processed=df_processed.rename(columns={"answer-items-text":"comment_texts"})
df_processed

,title,text,comment_texts
0,Попал в яму на Лада Приора,Здравствуйте. В общем день не задался. Ехал со...,['\n ...
1,Как подключить кнопку старт на Мазда 323 БФ?,"Уважаемые специалисты подскажите пожалуйста, м...",['\n ...
2,Какой артикул ремкомплекта заднего суппорта Сс...,Нужен артикул запчасти ремкомплект заднего лев...,['\n ...
3,Не горят контрольные лампы в Шевроле Авео,Не горят половина контрольных ламп. Менял щито...,['\n ...
4,Как проверить уровень в вариаторе JFO15E Нисса...,Здравствуйте! Как проверить уровень жидкости в...,['\n ...
...,...,...,...
46298,Де кнопка аварійного відключення палива у Рено...,Де знаходиться кнопка та шо отключає подачу па...,['\n ...
46299,Не включается вентилятор в Опель Кадет Е,Не включается вентилятор при нагреве двигателя,['\n ...
46300,Стоит ли ставить тормозные диски TRW DF4432S н...,Добрый день.\n\nПодыскиваю себе пару передних ...,['\n ...
46301,На панели где показывает передачи моргает в Хо...,Почему на панели где показывает скорости моргает?,"['Что мигает?', 'Что там на панели моргает?', ..."


In [259]:
df_processed=df_processed[df_processed["comment_texts"].isna().__invert__()]

df_processed

,title,text,comment_texts
0,Попал в яму на Лада Приора,Здравствуйте. В общем день не задался. Ехал со...,['\n ...
1,Как подключить кнопку старт на Мазда 323 БФ?,"Уважаемые специалисты подскажите пожалуйста, м...",['\n ...
2,Какой артикул ремкомплекта заднего суппорта Сс...,Нужен артикул запчасти ремкомплект заднего лев...,['\n ...
3,Не горят контрольные лампы в Шевроле Авео,Не горят половина контрольных ламп. Менял щито...,['\n ...
4,Как проверить уровень в вариаторе JFO15E Нисса...,Здравствуйте! Как проверить уровень жидкости в...,['\n ...
...,...,...,...
46298,Де кнопка аварійного відключення палива у Рено...,Де знаходиться кнопка та шо отключає подачу па...,['\n ...
46299,Не включается вентилятор в Опель Кадет Е,Не включается вентилятор при нагреве двигателя,['\n ...
46300,Стоит ли ставить тормозные диски TRW DF4432S н...,Добрый день.\n\nПодыскиваю себе пару передних ...,['\n ...
46301,На панели где показывает передачи моргает в Хо...,Почему на панели где показывает скорости моргает?,"['Что мигает?', 'Что там на панели моргает?', ..."


In [260]:
def extract_list(x):
    return re.findall(r"(?<=\')(.+?)(?=\')",x)

df_processed["comment_texts"]=df_processed["comment_texts"].apply(extract_list)

df_processed

C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/2456269486.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["comment_texts"]=df_processed["comment_texts"].apply(extract_list)


,title,text,comment_texts
0,Попал в яму на Лада Приора,Здравствуйте. В общем день не задался. Ехал со...,[\n ...
1,Как подключить кнопку старт на Мазда 323 БФ?,"Уважаемые специалисты подскажите пожалуйста, м...",[\n ...
2,Какой артикул ремкомплекта заднего суппорта Сс...,Нужен артикул запчасти ремкомплект заднего лев...,[\n ...
3,Не горят контрольные лампы в Шевроле Авео,Не горят половина контрольных ламп. Менял щито...,[\n ...
4,Как проверить уровень в вариаторе JFO15E Нисса...,Здравствуйте! Как проверить уровень жидкости в...,[\n ...
...,...,...,...
46298,Де кнопка аварійного відключення палива у Рено...,Де знаходиться кнопка та шо отключає подачу па...,[\n ...
46299,Не включается вентилятор в Опель Кадет Е,Не включается вентилятор при нагреве двигателя,[\n ...
46300,Стоит ли ставить тормозные диски TRW DF4432S н...,Добрый день.\n\nПодыскиваю себе пару передних ...,[\n ...
46301,На панели где показывает передачи моргает в Хо...,Почему на панели где показывает скорости моргает?,"[Что мигает?, , , Что там на панели моргает?, ..."


In [261]:
#isnan for nullable strings and lists
def isnan(value):
    try:
        return math.isnan(float(value))
    except:
        return False


def merge_title_text(x):
    title=x.title
    text=x.text

    if(title==text and title==""):
        return np.NaN
    if(isnan(title)):
        return text
    if(isnan(text)):
        return title


    if(title==text):
        return text
    res=text+" "+title

    if(re.match(r"^\s+$",res)):
        return np.NaN
    return res

def merge_comment_texts(x):
    ct=x.comment_texts
    if(isnan(ct)):
        return np.NaN

    if(len(ct)==0):
        return np.NaN
    
    res=""
    for i in range(len(ct)):
        to_add=re.sub(r"\n\s+|\\n\s+"," ",ct[i])
        res+=to_add+" "

    if(re.match(r"^\s+$",res)):
        return np.NaN
    return res

df_processed["problem"]=df_processed.apply(merge_title_text,axis=1)
df_processed["solution"]=df_processed.apply(merge_comment_texts,axis=1)

df_processed

C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/3990507068.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["problem"]=df_processed.apply(merge_title_text,axis=1)
C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/3990507068.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["solution"]=df_processed.apply(merge_comment_texts,axis=1)


,title,text,comment_texts,problem,solution
0,Попал в яму на Лада Приора,Здравствуйте. В общем день не задался. Ехал со...,[\n ...,Здравствуйте. В общем день не задался. Ехал со...,Можно проверить на шиномонтажке геометрию дис...
1,Как подключить кнопку старт на Мазда 323 БФ?,"Уважаемые специалисты подскажите пожалуйста, м...",[\n ...,"Уважаемые специалисты подскажите пожалуйста, м...",Всё сделали не так. У вас на кнопку не приход...
2,Какой артикул ремкомплекта заднего суппорта Сс...,Нужен артикул запчасти ремкомплект заднего лев...,[\n ...,Нужен артикул запчасти ремкомплект заднего лев...,Артикул D4632и направляющие
3,Не горят контрольные лампы в Шевроле Авео,Не горят половина контрольных ламп. Менял щито...,[\n ...,Не горят половина контрольных ламп. Менял щито...,"Тогда нужно проверить разъем, провода щитка п..."
4,Как проверить уровень в вариаторе JFO15E Нисса...,Здравствуйте! Как проверить уровень жидкости в...,[\n ...,Здравствуйте! Как проверить уровень жидкости в...,"Щуп он либо есть. Либо его нет, чтобы владеле..."
...,...,...,...,...,...
46298,Де кнопка аварійного відключення палива у Рено...,Де знаходиться кнопка та шо отключає подачу па...,[\n ...,Де знаходиться кнопка та шо отключає подачу па...,"Подушки сработали? Если да, то подачу топлива..."
46299,Не включается вентилятор в Опель Кадет Е,Не включается вентилятор при нагреве двигателя,[\n ...,Не включается вентилятор при нагреве двигателя...,"Проверьте предохранитель, далее сам вентилято..."
46300,Стоит ли ставить тормозные диски TRW DF4432S н...,Добрый день.\n\nПодыскиваю себе пару передних ...,[\n ...,Добрый день.\n\nПодыскиваю себе пару передних ...,Нормальный выбор тормозных дисков. TRW — это ...
46301,На панели где показывает передачи моргает в Хо...,Почему на панели где показывает скорости моргает?,"[Что мигает?, , , Что там на панели моргает?, ...",Почему на панели где показывает скорости морга...,"Что мигает? , Что там на панели моргает? , ..."


In [262]:
#drop unecessary columns
df_processed=df_processed.iloc[:,3:5]
df_processed

,problem,solution
0,Здравствуйте. В общем день не задался. Ехал со...,Можно проверить на шиномонтажке геометрию дис...
1,"Уважаемые специалисты подскажите пожалуйста, м...",Всё сделали не так. У вас на кнопку не приход...
2,Нужен артикул запчасти ремкомплект заднего лев...,Артикул D4632и направляющие
3,Не горят половина контрольных ламп. Менял щито...,"Тогда нужно проверить разъем, провода щитка п..."
4,Здравствуйте! Как проверить уровень жидкости в...,"Щуп он либо есть. Либо его нет, чтобы владеле..."
...,...,...
46298,Де знаходиться кнопка та шо отключає подачу па...,"Подушки сработали? Если да, то подачу топлива..."
46299,Не включается вентилятор при нагреве двигателя...,"Проверьте предохранитель, далее сам вентилято..."
46300,Добрый день.\n\nПодыскиваю себе пару передних ...,Нормальный выбор тормозных дисков. TRW — это ...
46301,Почему на панели где показывает скорости морга...,"Что мигает? , Что там на панели моргает? , ..."


In [263]:
#drop pairs problem-solution w/o problem or solution
df_processed=df_processed.dropna()
df_processed

,problem,solution
0,Здравствуйте. В общем день не задался. Ехал со...,Можно проверить на шиномонтажке геометрию дис...
1,"Уважаемые специалисты подскажите пожалуйста, м...",Всё сделали не так. У вас на кнопку не приход...
2,Нужен артикул запчасти ремкомплект заднего лев...,Артикул D4632и направляющие
3,Не горят половина контрольных ламп. Менял щито...,"Тогда нужно проверить разъем, провода щитка п..."
4,Здравствуйте! Как проверить уровень жидкости в...,"Щуп он либо есть. Либо его нет, чтобы владеле..."
...,...,...
46298,Де знаходиться кнопка та шо отключає подачу па...,"Подушки сработали? Если да, то подачу топлива..."
46299,Не включается вентилятор при нагреве двигателя...,"Проверьте предохранитель, далее сам вентилято..."
46300,Добрый день.\n\nПодыскиваю себе пару передних ...,Нормальный выбор тормозных дисков. TRW — это ...
46301,Почему на панели где показывает скорости морга...,"Что мигает? , Что там на панели моргает? , ..."


In [264]:
#get brand and model

def extract_list_no_empty(x):
    return re.findall(r"(?<=\')([\dа-яА-Яa-zA-Z\s\(\)\[\]]+?)(?=\')",x)
    
df_processed["breadcrumbs"]=df[df["breadcrumbs"].isna().__invert__()]["breadcrumbs"]
df_processed["breadcrumbs"]=df_processed["breadcrumbs"].apply(extract_list_no_empty)
df_processed

C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/668430338.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["breadcrumbs"]=df[df["breadcrumbs"].isna().__invert__()]["breadcrumbs"]
C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/668430338.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["breadcrumbs"]=df_processed["breadcrumbs"].apply(extract_list_no_empty)


,problem,solution,breadcrumbs
0,Здравствуйте. В общем день не задался. Ехал со...,Можно проверить на шиномонтажке геометрию дис...,"[Все вопросы, ВАЗ (Lada), Priora, Ходовая]"
1,"Уважаемые специалисты подскажите пожалуйста, м...",Всё сделали не так. У вас на кнопку не приход...,"[Все вопросы, Mazda, 323, 323 III (BF), Электр..."
2,Нужен артикул запчасти ремкомплект заднего лев...,Артикул D4632и направляющие,"[Все вопросы, SsangYong, Actyon, Actyon II, То..."
3,Не горят половина контрольных ламп. Менял щито...,"Тогда нужно проверить разъем, провода щитка п...","[Все вопросы, Chevrolet, Aveo, Электрика]"
4,Здравствуйте! Как проверить уровень жидкости в...,"Щуп он либо есть. Либо его нет, чтобы владеле...","[Все вопросы, Nissan, Qashqai, Qashqai I, Коро..."
...,...,...,...
46298,Де знаходиться кнопка та шо отключає подачу па...,"Подушки сработали? Если да, то подачу топлива...","[Все вопросы, Renault, Laguna, Laguna III, Эле..."
46299,Не включается вентилятор при нагреве двигателя...,"Проверьте предохранитель, далее сам вентилято...","[Все вопросы, Opel, Kadett, Kadett E, Охлажден..."
46300,Добрый день.\n\nПодыскиваю себе пару передних ...,Нормальный выбор тормозных дисков. TRW — это ...,"[Все вопросы, Honda, Accord, Accord VII, Тормо..."
46301,Почему на панели где показывает скорости морга...,"Что мигает? , Что там на панели моргает? , ...","[Все вопросы, Honda, Civic, Civic VIII, Салон]"


In [265]:
def return_with_check_brand(val):
    if(len(val)>=2):
        if(not re.match(r"двиг|электр",val[1].lower())):
            return val[1]

    return np.NaN


def return_with_check_model(val):
    if(len(val)>=4):
        if(not re.match(r"двиг|электр",val[2].lower())):
            return val[2]
    return np.NaN

df_processed["brand"]=df_processed["breadcrumbs"].apply(return_with_check_brand)
df_processed["model"]=df_processed["breadcrumbs"].apply(return_with_check_model)
df_processed

C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/3567200930.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["brand"]=df_processed["breadcrumbs"].apply(return_with_check_brand)
C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/3567200930.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["model"]=df_processed["breadcrumbs"].apply(return_with_check_model)


,problem,solution,breadcrumbs,brand,model
0,Здравствуйте. В общем день не задался. Ехал со...,Можно проверить на шиномонтажке геометрию дис...,"[Все вопросы, ВАЗ (Lada), Priora, Ходовая]",ВАЗ (Lada),Priora
1,"Уважаемые специалисты подскажите пожалуйста, м...",Всё сделали не так. У вас на кнопку не приход...,"[Все вопросы, Mazda, 323, 323 III (BF), Электр...",Mazda,323
2,Нужен артикул запчасти ремкомплект заднего лев...,Артикул D4632и направляющие,"[Все вопросы, SsangYong, Actyon, Actyon II, То...",SsangYong,Actyon
3,Не горят половина контрольных ламп. Менял щито...,"Тогда нужно проверить разъем, провода щитка п...","[Все вопросы, Chevrolet, Aveo, Электрика]",Chevrolet,Aveo
4,Здравствуйте! Как проверить уровень жидкости в...,"Щуп он либо есть. Либо его нет, чтобы владеле...","[Все вопросы, Nissan, Qashqai, Qashqai I, Коро...",Nissan,Qashqai
...,...,...,...,...,...
46298,Де знаходиться кнопка та шо отключає подачу па...,"Подушки сработали? Если да, то подачу топлива...","[Все вопросы, Renault, Laguna, Laguna III, Эле...",Renault,Laguna
46299,Не включается вентилятор при нагреве двигателя...,"Проверьте предохранитель, далее сам вентилято...","[Все вопросы, Opel, Kadett, Kadett E, Охлажден...",Opel,Kadett
46300,Добрый день.\n\nПодыскиваю себе пару передних ...,Нормальный выбор тормозных дисков. TRW — это ...,"[Все вопросы, Honda, Accord, Accord VII, Тормо...",Honda,Accord
46301,Почему на панели где показывает скорости морга...,"Что мигает? , Что там на панели моргает? , ...","[Все вопросы, Honda, Civic, Civic VIII, Салон]",Honda,Civic


In [266]:
df_processed["carinfo"]=df[df["carinfo"].isna().__invert__()]["carinfo"]

#year extraction
years=df_processed["carinfo"].astype("str").str.extract(r"(?P<year>(?<=\s)[12][0-9]{3}(?=[\s\-\—]))")
years=years.year

#Extract nan values
years=years.astype("float")
years=years.astype("Int64")

years[(years<1970)|(years>2022)]=np.NaN
display(years)

df_processed["year"]=years

C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/3803648408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["carinfo"]=df[df["carinfo"].isna().__invert__()]["carinfo"]


0        2007
1        1985
2        2010
3        2003
4        2007
         ... 
46298    2007
46299    1984
46300    2002
46301    2006
46302    2007
Name: year, Length: 45439, dtype: Int64

C:\Users\Oleg\AppData\Local\Temp/ipykernel_17964/3803648408.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed["year"]=years


In [267]:
print(df_processed.shape)
display(df_processed.dtypes)
df_processed

(45439, 7)


problem        object
solution       object
breadcrumbs    object
brand          object
model          object
carinfo        object
year            Int64
dtype: object

,problem,solution,breadcrumbs,brand,model,carinfo,year
0,Здравствуйте. В общем день не задался. Ехал со...,Можно проверить на шиномонтажке геометрию дис...,"[Все вопросы, ВАЗ (Lada), Priora, Ходовая]",ВАЗ (Lada),Priora,ВАЗ (Lada) Priora 2007 — н.в.,2007
1,"Уважаемые специалисты подскажите пожалуйста, м...",Всё сделали не так. У вас на кнопку не приход...,"[Все вопросы, Mazda, 323, 323 III (BF), Электр...",Mazda,323,Mazda 323 III (BF) 1985 — 1989,1985
2,Нужен артикул запчасти ремкомплект заднего лев...,Артикул D4632и направляющие,"[Все вопросы, SsangYong, Actyon, Actyon II, То...",SsangYong,Actyon,SsangYong Actyon II 2010 — н.в.,2010
3,Не горят половина контрольных ламп. Менял щито...,"Тогда нужно проверить разъем, провода щитка п...","[Все вопросы, Chevrolet, Aveo, Электрика]",Chevrolet,Aveo,Chevrolet Aveo I (T200/T250) 2003 — 2012,2003
4,Здравствуйте! Как проверить уровень жидкости в...,"Щуп он либо есть. Либо его нет, чтобы владеле...","[Все вопросы, Nissan, Qashqai, Qashqai I, Коро...",Nissan,Qashqai,Nissan Qashqai I 2007 — 2014,2007
...,...,...,...,...,...,...,...
46298,Де знаходиться кнопка та шо отключає подачу па...,"Подушки сработали? Если да, то подачу топлива...","[Все вопросы, Renault, Laguna, Laguna III, Эле...",Renault,Laguna,Renault Laguna III 2007 — 2015,2007
46299,Не включается вентилятор при нагреве двигателя...,"Проверьте предохранитель, далее сам вентилято...","[Все вопросы, Opel, Kadett, Kadett E, Охлажден...",Opel,Kadett,Opel Kadett E 1984 — 1994,1984
46300,Добрый день.\n\nПодыскиваю себе пару передних ...,Нормальный выбор тормозных дисков. TRW — это ...,"[Все вопросы, Honda, Accord, Accord VII, Тормо...",Honda,Accord,Honda Accord VII 2002 — 2007,2002
46301,Почему на панели где показывает скорости морга...,"Что мигает? , Что там на панели моргает? , ...","[Все вопросы, Honda, Civic, Civic VIII, Салон]",Honda,Civic,Honda Civic VIII 2006 — 2012,2006


In [268]:
df_processed=df_processed.drop(columns=["breadcrumbs","carinfo"])

In [269]:
#Now we can reset index
df_processed=df_processed.reset_index(drop=True)
df_processed

,problem,solution,brand,model,year
0,Здравствуйте. В общем день не задался. Ехал со...,Можно проверить на шиномонтажке геометрию дис...,ВАЗ (Lada),Priora,2007
1,"Уважаемые специалисты подскажите пожалуйста, м...",Всё сделали не так. У вас на кнопку не приход...,Mazda,323,1985
2,Нужен артикул запчасти ремкомплект заднего лев...,Артикул D4632и направляющие,SsangYong,Actyon,2010
3,Не горят половина контрольных ламп. Менял щито...,"Тогда нужно проверить разъем, провода щитка п...",Chevrolet,Aveo,2003
4,Здравствуйте! Как проверить уровень жидкости в...,"Щуп он либо есть. Либо его нет, чтобы владеле...",Nissan,Qashqai,2007
...,...,...,...,...,...
45434,Де знаходиться кнопка та шо отключає подачу па...,"Подушки сработали? Если да, то подачу топлива...",Renault,Laguna,2007
45435,Не включается вентилятор при нагреве двигателя...,"Проверьте предохранитель, далее сам вентилято...",Opel,Kadett,1984
45436,Добрый день.\n\nПодыскиваю себе пару передних ...,Нормальный выбор тормозных дисков. TRW — это ...,Honda,Accord,2002
45437,Почему на панели где показывает скорости морга...,"Что мигает? , Что там на панели моргает? , ...",Honda,Civic,2006


In [270]:
df_processed.to_csv("..\\data\\processed\\df_processed_etlib.csv")